## 教師データ収集用スクリプト

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [3]:
button_layout = widgets.Layout(width='128px', height='48px')
free_button = widgets.Button(description='forward', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='left', button_style='danger', layout=button_layout)
stop_button = widgets.Button(description='stop', button_style='success', layout=button_layout) 
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    print("saved free")
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    print("saved blocked")
    blocked_count.value = len(os.listdir(blocked_dir))

In [5]:
from jetbot import Robot
robot = Robot()

In [6]:
import time

isFree = False
isRunning = False
stepcount = 0

def set_forward(change):
    global stepcount, isFree, isRunning, robot
    stepcount = 0
    isRunning = True
    isFree = True
    robot.forward(0.4)

def set_left(change):
    global stepcount, isFree, isRunning, robot
    robot.stop()
    save_blocked()
    robot.left(0.4)
    stepcount = 0
    isRunning = True
    isFree = False
            

def update(change):
    global stepcount, isFree, isRunning, robot
    time.sleep(0.001)
    #blocked_count.value = stepcount
    if(isRunning):
        if(isFree):
            if(stepcount % 25 == 0):
                robot.stop()
                save_free()
                robot.forward(0.4)
        else:
            if(stepcount % 6 == 5):
                isRunning = False
                robot.stop()
        stepcount += 1
        
    
def stop(change):
    global isRunning, robot
    isRunning = False
    robot.stop()
    
free_button.on_click(set_forward)
blocked_button.on_click(set_left)
stop_button.on_click(stop)

In [7]:
display(image)
display(widgets.HBox([free_button, free_count]))
display(widgets.HBox([blocked_button, blocked_count]))
display(stop_button)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(button_style='success', description='stop', layout=Layout(height='48px', width='128px'), style=ButtonSt…

In [8]:
camera.observe(update, names='value')

In [9]:
camera.unobserve(update, names='value')

In [9]:
!zip -r -q dataset.zip dataset

In [ ]:
robot.stop()